# Advanced MPAS Analysis & Visualization with UXarray

<img src="path or URL to some visual here"
     width="30%"
     alt="MPAS advanced visual"
     align="right"
/>

### In this section, you'll learn:

* Utilizing the UXarray package to perform advanced analysis over MPAS data, such as cross-sections and zonal averages, etc.
* Using Matplotlib and hvPlot to visualize analysis.

### Related Documentation

* [URL title](URL)
* 

### Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| [UXarray](https://uxarray--1321.org.readthedocs.build/en/1321/index.html#) | Necessary  | |
| [SciPy](https://scipy.org/) | Helpful  | |
| [HoloViews](https://holoviews.org/) | Helpful | |


**Time to learn**: 30 minutes?

-----

## Import packages


In [ ]:
%%time 

# autoload external python modules if they changed
%load_ext autoreload
%autoreload 2

# add ../funcs to the current path
import sys, os
sys.path.append(os.path.join(os.getcwd(), "..")) 

import warnings
import math

import cartopy.crs as ccrs
import geoviews as gv
import geoviews.feature as gf
import matplotlib.pyplot as plt

import s3fs

import geopandas as gp
import numpy as np
import uxarray as ux
import xarray as xr

#### UXarray and hvPlot
`hvPlot` is a high-level API built on `HoloViews` that provides interactive plots. It is integrated with `UXarray` with a `.hvplot()` similar to `.plot()` in `Pandas` library, but will produce more interactive figures. Import the relevant packages if you want to use the `hvplot` API.

In [ ]:
%%time 
import holoviews as hv
import hvplot.xarray
from holoviews import opts

hv.extension("bokeh")
hv.extension("matplotlib")


## Configure visualization tools

In [ ]:
# common border lines
coast_lines = gf.coastline(projection=ccrs.PlateCarree(), line_width=1, scale="50m")
state_lines = gf.states(projection=ccrs.PlateCarree(), line_width=1, line_color='gray', scale="50m")

## Retrieve/load  MPAS/JEDI data
The example MPAS/JEDI data are stored at [jetstream2](https://par.nsf.gov/biblio/10296117-jetstream2-accelerating-cloud-computing-via-jetstream). We need to retreive those data first.   
There are two ways to retrieve MPAS data:
- 1. Download all example data from JetStream2 to local and them load them locally. This approach allows downloading the data once per machine and reuse it in notebooks.
- 2. Stream the JetStream2 S3 objects on demand. In this case, each notebook (including restarting a notebook) will retrieve the required data separately as needed.

In [ ]:
# choose the data_load_method, check the above cell for details. Default to method 1, i.e. download once and reuse it in multiple notebooks
data_load_method = 2  # 1 or 2

### Method 1: Download all example data once and reuse it in mulptile notebooks

In [ ]:
%%time
local_dir="/tmp"

if data_load_method == 1 and not os.path.exists(local_dir + "/conus12km/bkg/mpasout.2024-05-06_01.00.00.nc"):
    jetstream_url = 'https://js2.jetstream-cloud.org:8001/'
    fs = s3fs.S3FileSystem(anon=True, asynchronous=False,client_kwargs=dict(endpoint_url=jetstream_url))
    conus12_path = 's3://pythia/mpas/conus12km'
    fs.get(conus12_path, local_dir, recursive=True)
    print("Data downloading completed")
else:
    print("Skip..., either data is available in local or data_load_method is NOT 1")

In [ ]:
# Set file path
if data_load_method == 1:
    grid_file = local_dir + "/conus12km/conus12km.invariant.nc_L60_GFS"
    ana_file = local_dir + "/conus12km/bkg/mpasout.2024-05-06_01.00.00.nc"
    bkg_file = local_dir + "/conus12km/ana/mpasout.2024-05-06_01.00.00.nc"
    # jdiag_file = local_dir + "/conus12km/jdiag_aircar_t133.nc"  #q133.nc or uv233.nc

### Method 2: Stream the JetStream2 S3 objects on demand

In [ ]:
%%time
if data_load_method == 2:
    jetstream_url = 'https://js2.jetstream-cloud.org:8001/'
    fs = s3fs.S3FileSystem(anon=True, asynchronous=False,client_kwargs=dict(endpoint_url=jetstream_url))
    conus12_path = 's3://pythia/mpas/conus12km'
    
    grid_url=f"{conus12_path}/conus12km.invariant.nc_L60_GFS"
    bkg_url=f"{conus12_path}/bkg/mpasout.2024-05-06_01.00.00.nc"
    ana_url=f"{conus12_path}/ana/mpasout.2024-05-06_01.00.00.nc"
    # jdiag_url=f"{conus12_path}/jdiag_aircar_t133.nc"
    
    grid_file = fs.open(grid_url)
    ana_file = fs.open(ana_url)
    bkg_file = fs.open(bkg_url)
    # jdiag_file = fs.open(jdiag_url)
else:
    print("Skip..., data_load_method is NOT 2")

:::{warning}
Depending on the network conditions, loading the data can take a few minutes.
:::

### Loading the data into UXarray datasets

We use the UXarray data structures for working with the data. This package supports data defined over unstructured grid and provides utilities for modifying and visualizing it. The available fucntionality are discussed in [`UxDataset` documentation](https://uxarray.readthedocs.io/en/latest/generated/uxarray.UxDataset.html#uxarray.UxDataset).    
For more information about the `UXarray` and unstructured grid, please go to [Working with unstructured grids with UXarray](https://uxarray.readthedocs.io/en/v2023.08.0/examples/001-working-with-unstructured-grids.html).

In [ ]:
uxds_a = ux.open_dataset(grid_file, ana_file)
uxds_b = ux.open_dataset(grid_file, bkg_file)

# We will extract the potential temperature `theta` from the analysis data from MPAS and convert it from Kelvin to Celsius.
uxvar = uxds_a['theta'] - 273.15   ## Kelvin to Celsius

### Indexing and selecting parameters
Also, for simplicity, let's focus on the time indices of 0. 

In [ ]:
i_time = 0  # `Time` index
uxvar = uxvar.isel(Time=0)

## Vertical cross section

Based on `UXarray`, we will generate cross-sections:

- along an arbitrary great‑circle arcs (GCAs) between two point over the sphere surface.
- along a constant longitude or latitude line and

We will also mark the lines or arcs over a map.

:::{hint}
We assume that you have already gone over the previous section, **[Basic MPAS analysis and visualization with UXarray](mpas-basic)**. If not and if you need to learn about basic indexing, selecting, and generating horizontal figures of variables, we recommend to check that section first.
:::

### Plot Parameters

In [ ]:
# for matplotlib
tick_stride=10

# for hvplot

### Random Great Circle Arc (GCA)

Let us use UXarray's vertical cross-section function to get a cross-section over a great circle arc:

In [ ]:
%%time

start_point = (-60.,20.) # (start_lon, start_lat)
end_point = (-40., 50.) # (end_lon,end_lat)
step_between_points = 100

cross_section_gca = uxvar.cross_section(start=start_point, end=end_point, steps=step_between_points)
cross_section_gca

In [ ]:
hlabelticks = [
    f"{abs(lat):.1f}°{'N' if lat >= 0 else 'S'}\n{abs(lon):.1f}°{'E' if lon >= 0 else 'W'}"
    for lat, lon in zip(cross_section_gca['lat'], cross_section_gca['lon'])
]

UXarray's cross-section returns an `xarray.DataArray` that can then be plotted:

In [ ]:
%matplotlib inline


fig= plt.figure(figsize=(8,3))
gs= fig.add_gridspec(1,1)
ax = fig.add_subplot(gs[0,0])
cf=ax.contourf(cross_section_gca.transpose(),cmap='Reds',extend='both')

ax.set_xticks(cross_section_gca['steps'][::tick_stride])
ax.set_xticklabels(hlabelticks[::tick_stride])

### Constant Latitude

In [ ]:
lat=43.3
step_between_points = 100

cross_section_lat = uxvar.cross_section(lat=lat, steps=step_between_points)

hlabelticks = [
    f"{abs(lon):.1f}°{'E' if lon >= 0 else 'W'}" for lon in cross_section_lat['lon']
]

%matplotlib inline
fig= plt.figure(figsize=(8,3))
gs= fig.add_gridspec(1,1)
ax = fig.add_subplot(gs[0,0])
cf=ax.contourf(cross_section_lat.transpose(),cmap='Reds',extend='both')


ax.set_xticks(cross_section_lat['steps'][::tick_stride])
ax.set_xticklabels(hlabelticks[::tick_stride])

In [ ]:

cross_section_lat.assign_coords({'steps':range(len(hlabelticks)),'nVertLevels':range(cross_section_lat.shape[1])}).hvplot.contourf(
        x=cross_section_lat.dims[0],y=cross_section_lat.dims[1],
        ) 

### COnst

In [ ]:
lon=-83.3
cross_section_lon = uxvar.cross_section(lon=lon, steps=step_between_points)

hlabelticks = [
    f"{abs(lat):.1f}°{'N' if lat >= 0 else 'S'}" for lat in cross_section_lon['lat']
]

%matplotlib inline
fig= plt.figure(figsize=(8,3))
gs= fig.add_gridspec(1,1)
ax = fig.add_subplot(gs[0,0])
cf=ax.contourf(cross_section_lon.transpose(),cmap='Reds',extend='both')

ax.set_xticks(cross_section_lon['steps'][::tick_stride])
ax.set_xticklabels(hlabelticks[::tick_stride])

In [ ]:
cross_section_lon = uxvar.cross_section(lon=-60., steps=100)
cross_section_lon

In [ ]:
cross_section_lat = uxvar.cross_section(lat=40., steps=100)
cross_section_lat